In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#library to install
!pip install datasets evaluate peft sacrebleu bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does no

In [ ]:
#libary to import
from datasets import load_dataset, DatasetDict
from transformers import AutoModelForSeq2SeqLM, NllbTokenizerFast, set_seed, BertForSequenceClassification
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from peft import LoraConfig, TaskType
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
from tqdm import tqdm
import numpy as np
import evaluate
metric = evaluate.load("sacrebleu")

In [ ]:
tokenizer = NllbTokenizerFast.from_pretrained(
    "facebook/nllb-200-distilled-1.3B", src_lang="eng_Latn", tgt_lang="twi_Latn",
)
model = AutoModelForSeq2SeqLM.from_pretrained(
    "facebook/nllb-200-distilled-1.3B",
    load_in_4bit=True,
    device_map='auto')
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1, target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin:   0%|          | 0.00/5.48G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

trainable params: 2,359,296 || all params: 1,372,997,632 || trainable%: 0.1718


In [ ]:
# Load the CSV file
dataset = load_dataset('csv', data_files='/content/drive/My Drive/DLI_Hackathon/LLM_Hack/Train.csv')

# Split the dataset into training and validation sets (90% train, 10% validation)
train_valid_split = dataset['train'].train_test_split(test_size=0.1)

# Combine the split datasets into a single DatasetDict
dataset_dict = DatasetDict({
    'train': train_valid_split['train'],
    'validation': train_valid_split['test']
})

# You now have a combined DatasetDict with both train and validation sets
print(dataset_dict)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'English', 'Twi'],
        num_rows: 4320
    })
    validation: Dataset({
        features: ['id', 'English', 'Twi'],
        num_rows: 480
    })
})


In [ ]:
#tokenize the dataset
def preprocess_function(examples):
    model_inputs = tokenizer(examples['English'], text_target=examples['Twi'], max_length=128, truncation=True)
    return model_inputs

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
tokenized_datasets = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/4320 [00:00<?, ? examples/s]

Map:   0%|          | 0/480 [00:00<?, ? examples/s]

In [ ]:
#processing the data for use
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
#model taining parameters for sequence to sequence model
training_args = Seq2SeqTrainingArguments(
    output_dir="outputs",
    evaluation_strategy="steps",
    save_strategy='steps',
    logging_steps=500,
    eval_steps=1_000,
    save_steps=1_000,
    learning_rate=1e-3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=24,
    auto_find_batch_size=True,
    weight_decay=0.01,
    num_train_epochs=3,
    predict_with_generate=True,
    load_best_model_at_end=True,
    save_total_limit=1,
    metric_for_best_model='eval_bleu',
    ddp_find_unused_parameters=False
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:435: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss,Validation Loss,Bleu,Gen Len
1000,1.603200,1.602539,23.681100,27.943800


TrainOutput(global_step=1620, training_loss=1.6267680603780865, metrics={'train_runtime': 1431.795, 'train_samples_per_second': 9.052, 'train_steps_per_second': 1.131, 'total_flos': 3539467199250432.0, 'train_loss': 1.6267680603780865, 'epoch': 3.0})

In [ ]:
#saving the trained model
trainer.model.save_pretrained("/content/drive/My Drive/DLI_Hackathon/LLM_Hack/qlora-nllb-en-twi")

In [ ]:
#setting batch and model evaluation
import torch

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """
    Split the dataset into smaller batches for processing.

    Args:
        list_of_elements (list): The list to be split into batches.
        batch_size (int): The size of each batch.

    Yields:
        list: A batch of elements from the input list.
    """
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

# Modify the predict_on_test_ds function to return a flat list of predictions
def predict_on_test_ds(
    dataset,
    model,
    tokenizer,
    batch_size=16,
    device="cuda" if torch.cuda.is_available() else "cpu",
    input_text="English",
):

"""
    Calculate evaluation metrics on the test dataset.

    Args:
        dataset (Dataset): The dataset to evaluate on.
        metric (Metric): The metric to use for evaluation.
        model (PreTrainedModel): The model to use for generation.
        tokenizer (PreTrainedTokenizer): The tokenizer for the model.
        batch_size (int): The batch size for processing.
        device (str): The device to use for computation.
        column_text (str): The name of the column containing the input text.
        column_summary (str): The name of the column containing the target summary.

    Returns:
        dict: A dictionary containing the computed metrics.
    """


    model.to(device)
    article_batches = list(generate_batch_sized_chunks(dataset[input_text], batch_size))
    target_translations = []

    for article_batch in tqdm(article_batches, total=len(article_batches)):
        inputs = tokenizer(
            article_batch,
            max_length=512,
            truncation=True,
            padding=True,
            return_tensors="pt",
        ).to(device)

        translations = model.generate(
            input_ids=inputs["input_ids"].to(device),
            attention_mask=inputs["attention_mask"].to(device),
        )

        decoded_translations = [
            tokenizer.decode(
                s, skip_special_tokens=True, clean_up_tokenization_spaces=True
            )
            for s in translations
        ]

        target_translations.extend(decoded_translations)  # Use extend to flatten the list

    return target_translations

In [ ]:
# Load the CSV file into a pandas DataFrame
from datasets import Dataset
test_df = pd.read_csv('/content/drive/My Drive/DLI_Hackathon/LLM_Hack/Test.csv', index_col=False)

# Convert the pandas DataFrame to a Huggingface Dataset
test_data = Dataset.from_pandas(test_df)

# Generate predictions
predictions = predict_on_test_ds(test_data, trainer.model, tokenizer, batch_size=48, device="cuda", input_text="English")

# Add the predictions as a new column to the original dataframe
test_df['Predicted Translation'] = predictions

100%|██████████| 67/67 [11:01<00:00,  9.87s/it]

Predicted results saved successfully!


In [ ]:
# Save the dataframe with the original text and predictions into a new CSV file
test_df[['id', 'Predicted Translation']].to_csv('/content/drive/My Drive/DLI_Hackathon/LLM_Hack/Submission.csv', index=False)

print("Predicted results saved successfully!")

Predicted results saved successfully!
